In [22]:
import re
from IPython.display import display

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup


import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
# from tqdm.notebook import tqdm
from tqdm import tqdm

pd.set_option ('display.max_colwidth', None)
pd.set_option ('display.max_columns', None)

nltk.download ('punkt')
nltk.download ('stopwords')
nltk.download ('wordnet')

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

[nltk_data] Downloading package punkt to /Users/kunmi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/kunmi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kunmi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv ('../Data/NG_ELECTION_TWEETS.csv')
df.head (3)

,Unnamed: 0,id,text,created_at,author_id,retweet_count,reply_count,like_count,quote_count,withheld
0,0,1507404536511565825,"RT @AtedoPeterside: Nobody ""says"" it better than our GoAmbassador @mrmacaronii. Take matters into your own hands by acquiring your PVC now.…",2022-03-25T17:10:22.000Z,1220591726731124736,24,0,0,0,NaN
1,1,1507403883924103170,RT @1dernet: Getting Our PVC And Registering To Vote Come 2023 Is The Nonviolent Part To Take.\n#GetYourPVC \n@DeleFarotimi \n@Tsngcampaign ht…,2022-03-25T17:07:46.000Z,1169272207014543360,19,0,0,0,NaN
2,2,1507403359040544775,RT @Chude__: Hear what late Dora Akunyeli said about Peter Obi.. \n\n#GetYourPVC\n#Competense2023\n#peterObi2023 https://t.co/L0u1FqY4dB,2022-03-25T17:05:41.000Z,1387165423083433985,72,0,0,0,NaN


In [3]:
df.shape

(23437, 10)

In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer ()

def get_sentiment (tweet):
  polarity_score = analyzer.polarity_scores (tweet) ['compound']
  sentiment      = 'Positive' if polarity_score >= 0.05 \
              else 'Neutral'  if polarity_score > -0.05 else 'Negative'
  return sentiment

df ['Sentiment'] = df ['text'].map (get_sentiment)

In [5]:
df.to_csv ('../Data/preprocessed/NG_ELECTION_TWEETS_LABELLED.csv', index=False)

In [6]:
df.head (2)

,Unnamed: 0,id,text,created_at,author_id,retweet_count,reply_count,like_count,quote_count,withheld,Sentiment
0,0,1507404536511565825,"RT @AtedoPeterside: Nobody ""says"" it better than our GoAmbassador @mrmacaronii. Take matters into your own hands by acquiring your PVC now.…",2022-03-25T17:10:22.000Z,1220591726731124736,24,0,0,0,NaN,Positive
1,1,1507403883924103170,RT @1dernet: Getting Our PVC And Registering To Vote Come 2023 Is The Nonviolent Part To Take.\n#GetYourPVC \n@DeleFarotimi \n@Tsngcampaign ht…,2022-03-25T17:07:46.000Z,1169272207014543360,19,0,0,0,NaN,Neutral


In [7]:
df.Sentiment.nunique ()

3

In [8]:
df.Sentiment.value_counts ()

Positive    8747
Neutral     8573
Negative    6117
Name: Sentiment, dtype: int64

In [9]:
def custom_cleaner (tweet):
  soup    = BeautifulSoup (tweet, 'lxml') # removing HTML entities such as '&amp', '&quot', '&gt'; lxml is the html parser
  souped  = soup.get_text ()
  link_re = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
  re_ = re.sub (link_re, "", souped)
  re_ = re.sub (r"#\S*", "", re_)       # Replace all hashtags with an empty string
  re_ = re.sub ("RT @\S*", " ", re_)    # Removing user names
  re_ = re.sub (r"[^A-Za-z]+", " ", re_) # sustituting any non-alphabetic character that repeats one or more time

  tokens     = nltk.word_tokenize (re_)
  lower_case = [t.lower () for t in tokens]

  stop_words      = set (stopwords.words ('english'))
  filtered_result = list (filter (lambda l: l not in stop_words, lower_case))

  wordnet_lemmatizer = WordNetLemmatizer ()
  lemmas             = [wordnet_lemmatizer.lemmatize (t) for t in filtered_result]

  return lemmas

In [23]:
tqdm.pandas ()

for row, col  in tqdm(df['text'].iteritems (), total=df.shape[0]):
  df.loc[row, 'cleaned_tweet'] =  ' '.join (custom_cleaner (col))

100%|██████████| 18301/18301 [00:13<00:00, 1359.26it/s]


In [11]:
df.head (3)

,Unnamed: 0,id,text,created_at,author_id,retweet_count,reply_count,like_count,quote_count,withheld,Sentiment,cleaned_tweet
0,0,1507404536511565825,"RT @AtedoPeterside: Nobody ""says"" it better than our GoAmbassador @mrmacaronii. Take matters into your own hands by acquiring your PVC now.…",2022-03-25T17:10:22.000Z,1220591726731124736,24,0,0,0,NaN,Positive,nobody say better goambassador mrmacaronii take matter hand acquiring pvc
1,1,1507403883924103170,RT @1dernet: Getting Our PVC And Registering To Vote Come 2023 Is The Nonviolent Part To Take.\n#GetYourPVC \n@DeleFarotimi \n@Tsngcampaign ht…,2022-03-25T17:07:46.000Z,1169272207014543360,19,0,0,0,NaN,Neutral,getting pvc registering vote come nonviolent part take delefarotimi tsngcampaign ht
2,2,1507403359040544775,RT @Chude__: Hear what late Dora Akunyeli said about Peter Obi.. \n\n#GetYourPVC\n#Competense2023\n#peterObi2023 https://t.co/L0u1FqY4dB,2022-03-25T17:05:41.000Z,1387165423083433985,72,0,0,0,NaN,Neutral,hear late dora akunyeli said peter obi


In [19]:
df   = df [[len (i.split (" ")) >= 5 for i in df.cleaned_tweet]] 
data = df['cleaned_tweet']

In [20]:
df.shape

(18301, 12)

In [ ]:
df [['Sentiment', 'cleaned_tweet']].to_excel ('../Data/preprocessed/NG_ELECTION_TWEETS_CLEANED.xlsx',index=False)

In [15]:
df [['Sentiment', 'text']].to_excel ('../Data/preprocessed/NG_ELECTION_TWEETS_LABELLED.xlsx', index=False)